# Incorporate proposed roads into current roads surface

This notebook merges each of the new speed rasters to the "main" friction surface for the analysis extent area, generating a new friction surface specific to that road. We use that friction surface to generate accessibility surfaces specific to that road

In [1]:
import os, sys
from datetime import date

import numpy as np
import re

import rasterio
from rasterio import features, transform
from rasterio.merge import merge as merge
from rasterio.transform import Affine
from rasterio.io import MemoryFile

import common_rasterio_ops as rast_ops

import rio_cogeo
from rio_cogeo.cogeo import cog_translate

import pandas as pd
import geopandas as gpd

import shapely
from shapely.geometry import shape, box, MultiPoint, Point, Polygon

import skimage.graph as graph
sys.path.append('../../src/')
import GOSTnets.GOSTNets as gn
import GOSTNets_Raster.src.GOSTNets_Raster.market_access as ma
from gostrocks.src.GOSTRocks.misc import tPrint

## Setup

Today's date

In [2]:
today = date.today().strftime("%y%m%d")

In [3]:
data_date = '220118'

Desired resolution of rasters

In [4]:
res = '31m'

Directories

In [5]:
geo_dir = r'P:\PAK\GEO'
data_dir = r'../../data'

rd_dir = r'roads'
dest_dir = r'destinations'
fric_dir = r'friction'
acc_dir = r'access'

Projections

In [6]:
# change this to whatever the desired output projection is
dest_crs = 'EPSG:32642'

Load in KP as clipping object

In [7]:
aoi = gpd.read_file(os.path.join(geo_dir,'Boundaries/OCHA/pak_admbnda_adm1_ocha_pco_gaul_20181218.shp'))
aoi = aoi[aoi['ADM1_EN'] == 'Khyber Pakhtunkhwa']
aoi = aoi.to_crs(dest_crs)

# Buffer the polygon by 20km so we take in nearby markets and roads that may be used
aoi.geometry = aoi.buffer(20000)

Destination file

In [ ]:
# Use destinations prepared previously. This assumes they are in WGS84 and need to be reprojected to the project's metric CRS

dest_fils = {
    re.findall(r'KP_(.*?).gpkg',fil)[0]: gpd.clip(gpd.read_file(os.path.join(data_dir,dest_dir,fil))\
                                                  .set_crs(4326).to_crs(dest_crs),aoi) \
    for fil in os.listdir(os.path.join(data_dir,dest_dir)) if fil.endswith(".gpkg")
}

In [ ]:
len(dest_fils)

24

**Functions to process roads vectors into rasters**

In [ ]:
# Define a function to turn the prepared vector into an appropriately sized numpy array using the generator from `get_raster_params`

def rasterize_vector(vector,base_rast_raw,speed_column):

    # Run the above function to get all the necessary inputs for rasterization: parameters + the vector generator
    
    bds, ht, wth, src_tform, dst_tform, shapes, scale_factor_x, scale_factor_y = rast_ops.get_raster_params(vector,base_rast_raw,speed_column)
    
    # Downsample while rasterizing, saving us expensive resampling in memory.
    
    road_rast = features.rasterize(shapes,\
                  out_shape = (int(ht / scale_factor_y),\
                               int(wth / scale_factor_x)),\
                  transform=dst_tform,
                  all_touched=True,
                  dtype = np.float64)
    
    return road_rast

# Define a function to export a road's numpy array to a GeoTIFF for spatial analysis / visualization purposes

def export_rast(vector, fin_rast):

    # calculate naming variables
    
    sn = vector.SN.item()
    district = vector.District.item()
    season = re.findall(r'(.*?)_speed',speed_column)[0]
    
    fric_out_name = os.path.basename(f'{sn}_{district}_{season}_friction.tif')
    
    # Must re-run this to get correct parameters for the profile creation
    bds, ht, wth, src_tform, dst_tform, shapes, scale_factor_x, scale_factor_y = rast_ops.get_raster_params(vector,base_rast_raw,speed_column)

    # Create metadata, including transformation, for the export
    fin_profile = {
        "driver": "GTiff",
        "dtype": "float32",
        "crs": {'init': dest_crs},
        "height": int(ht / scale_factor_y),
        "width": int(wth / scale_factor_x),
        "count": 1,
        "nodata": 0,
        "transform": dst_tform,
        "compress": 'LZW'
    }
    
    # compute friction surface
    fric_fin_rast = (1 / fin_rast) / (1000 / dst_tform.a)
    fric_fin_rast = np.where(fric_fin_rast > 1, 1, fric_fin_rast)
    fric_fin_rast = fric_fin_rast.astype(np.float32)
    
    # Export friction raster
    with rasterio.open(
        os.path.join(data_dir,fric_dir,f'proposed_roads//{res}//raw',fric_out_name), 'w',**fin_profile) as dst:
        dst.write(fric_fin_rast, indexes=1)
        
# batch these into one function

def rasterization_routine(vector,base_rast_raw,speed_column):
    
    fin_rast = rasterize_vector(vector,base_rast_raw,speed_column)
    
    export_rast(vector, fin_rast)

## Processing roads vectors into friction rasters

### Generate and export small area friction rasters for each proposed road 

Desired output resolution

In [ ]:
res = '31m'
# res = '100m'

Roads

In [ ]:
prop_rds = gpd.read_file(os.path.join(data_dir,rd_dir,f'Proposed_final//Proposed_roads_processed_{data_date}.gpkg'),driver="GPKG")

Populate lists to loop over

In [14]:
speed_cols = ['upgrade_dry_speed', 'upgrade_msn_speed', 'upgrade_winter_speed']

In [15]:
fric_tifs = sorted([os.path.join(data_dir,fric_dir,f'{res}//driving_only',file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir,f'{res}//driving_only')) \
            if file.endswith(".tif")]) # currently set up for walking rasters, change to whatever you need

In [16]:
fric_tifs

['../../data\\friction\\31m//driving_only\\KP_friction_dry_211020_31m_masked.tif',
 '../../data\\friction\\31m//driving_only\\KP_friction_msn_211020_31m_masked.tif',
 '../../data\\friction\\31m//driving_only\\KP_friction_winter_211020_31m_masked.tif']

Loop

In [17]:
# Loop over every speed_col for every friction tif and export a small-scale friction raster

for fric in fric_tifs:   
    
    # fric_variable
    
    fric_name = os.path.basename(fric).replace('.tif','')
    
    # Load in the base speed raster we are merging unto so we can match up exactly to its grid and cell size

    with rasterio.open(
            os.path.join(data_dir,fric_dir,fric), 'r') as base_rast_raw:
        base_idx = base_rast_raw.index
        base_profile = base_rast_raw.meta.copy()
        base_tform = base_rast_raw.transform
        
    # Specify speed value being burned into the raster
    
    for speed_column in speed_cols:

        tPrint(f'{fric_name}, {speed_column}')
        
        # Need to reorder the datafram least to highest, so that the generator and therefore speed values are written similarly, 
        # This has the ultimate consequence that higher values overwrite lower values where they overlap
        prop_rds = prop_rds.sort_values(by=speed_column)
        
        # rd-lst = [file for file in os.listdir(os.path.join(data_dir,rd_dir)) if file.endswith(".shp")]
        rds_lst = [gpd.GeoDataFrame(road).T for idx, road in prop_rds.iterrows()]
        
        for idx, rd in enumerate(rds_lst):

            rasterization_routine(rd,base_rast_raw,speed_column)
        

12:01:02	KP_friction_dry_211020_31m_masked, upgrade_dry_speed


C:\Users\wb467985\AppData\Local\Temp/ipykernel_36760/3940569224.py:49: RuntimeWarning: divide by zero encountered in true_divide
  fric_fin_rast = (1 / fin_rast) / (1000 / dst_tform.a)


12:01:24	KP_friction_dry_211020_31m_masked, upgrade_msn_speed
12:01:45	KP_friction_dry_211020_31m_masked, upgrade_winter_speed
12:02:07	KP_friction_msn_211020_31m_masked, upgrade_dry_speed
12:02:29	KP_friction_msn_211020_31m_masked, upgrade_msn_speed
12:02:50	KP_friction_msn_211020_31m_masked, upgrade_winter_speed
12:03:10	KP_friction_winter_211020_31m_masked, upgrade_dry_speed
12:03:30	KP_friction_winter_211020_31m_masked, upgrade_msn_speed
12:03:51	KP_friction_winter_211020_31m_masked, upgrade_winter_speed


## Incorporate existing proposed roads into friction surface

Populate lists to loop over

In [13]:
seasons = ['dry','msn','winter']

Assumes that master friction surfaces have been stored in a "{res}\\current" folder

In [14]:
fric_tifs = sorted([os.path.join(data_dir,fric_dir,f'{res}//current',file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir,f'{res}//current')) \
            if file.endswith(".tif")]) # currently set up for walking rasters, change to whatever you need

#### Incorporation loop

In [ ]:
# create export dir if missing
if os.path.exists(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed')) == False:
    os.mkdir(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed'))
else:
    None

In [19]:
for seas in seasons:
    
    # name variables
    speed_column = f'current_{seas}_speed'
    
    base_rast_name = f'KP_friction_{seas}_{data_date}_{res}_masked'
    base_rast_pth = base_rast_name + '.tif'
    prop_rast_pth = base_rast_name + '_final.tif'
    
    # load in base raster
    base_rast = rasterio.open(os.path.join(data_dir,fric_dir,f'{res}//current',base_rast_pth),'r')
    base_profile = base_rast.profile
    base_tform = base_rast.transform
    base_profile.update({'dtype':'float32'})
    
    # create a generator containing geometry, value pairs for prop_rds
    prop_rds_shapes = ((geom,speed) for geom, speed in zip(prop_rds.geometry,prop_rds[speed_column]))

    # This will give the raster the size and dimensions of the base friction raster
    prop_rast_fin = features.rasterize(prop_rds_shapes,\
                      out_shape = (base_profile['height'],\
                                   base_profile['width']),\
                      transform=base_profile['transform'],
                      all_touched=True,
                      fill=-1,
                      dtype = np.float32)
    
    # transform into friction values
    prop_rast_fin = (1 / prop_rast_fin) / (1000 / base_tform.a)
    
    # merge with existing friction surface
    prop_rast_fin = np.where(prop_rast_fin > 0, prop_rast_fin, base_rast.read(1))
    prop_rast_fin = prop_rast_fin.astype(np.float32)
        
    # export
    
    with rasterio.open(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed',prop_rast_pth),'w',**base_profile) as dst:
        dst.write(prop_rast_fin,indexes=1)

### Convert all friction GeoTIFFs to Cloud Optimized GeoTIFFs

Convert all GeoTIFFs to COG GeoTIFFs. COGs are more efficiently tiled and compressed GeoTIFFs optimized to be served out of cloud storage environments. They are notably slimmer than the GeoTIFFs generated by MCP and necessary for later Dask operations

In [58]:
# import glob
# # cleanup code if anything is misnamed

# for f in glob.glob(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed','*')):
#     os.rename(f, f.replace('walkchild', 'childwalk'))

In [59]:
noncog_tifs = sorted([os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed',file) \
            for file \
            in os.listdir(os.path.join(data_dir,fric_dir,f'{res}//current_w_proposed')) \
            if file.endswith(".tif")])

In [60]:
noncog_tifs[:3]

['../../data\\friction\\31m//current_w_proposed\\KP_friction_dry_211019_31m_masked_final.tif',
 '../../data\\friction\\31m//current_w_proposed\\KP_friction_msn_211019_31m_masked_final.tif',
 '../../data\\friction\\31m//current_w_proposed\\KP_friction_winter_211019_31m_masked_final.tif']

In [61]:
for noncog_lyr in noncog_tifs:
    cog_lyr = str.replace(noncog_lyr,'.tif','') + '_COG.tif'
    
    print(cog_lyr)
    !rio cogeo create {noncog_lyr} {cog_lyr}

../../data\friction\31m//current_w_proposed\KP_friction_dry_211019_31m_masked_final_COG.tif
../../data\friction\31m//current_w_proposed\KP_friction_msn_211019_31m_masked_final_COG.tif


Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/current_w_proposed/KP_friction_dry_211019_31m_masked_final.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\current_w_proposed\KP_friction_dry_211019_31m_masked_final_COG.tif


../../data\friction\31m//current_w_proposed\KP_friction_winter_211019_31m_masked_final_COG.tif


Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/current_w_proposed/KP_friction_msn_211019_31m_masked_final.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\current_w_proposed\KP_friction_msn_211019_31m_masked_final_COG.tif
Reading input: P:/PAK/Code/kpgit_robert/data/friction/31m/current_w_proposed/KP_friction_winter_211019_31m_masked_final.tif
Adding overviews...
Updating dataset tags...
Writing output to: \\sarpov\sarpov\PAK\Code\kpgit_robert\data\friction\31m\current_w_proposed\KP_friction_winter_211019_31m_masked_final_COG.tif
